# Training module

In [97]:
#import library
import csv
import pandas as pd
import pickle 
import numpy as np
import cv2
import mediapipe as mp
import time
import shutil
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score 

# Test Camera

In [98]:
# Grabbing the Holistic Model from Mediapipe and
mp_holistic = mp.solutions.holistic

# Initializing the Model
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

In [99]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
    
    # resizing the frame for better view
    frame = cv2.resize(frame, (880,720))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime
    
    
    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
 
    # Display the resulting image
    cv2.imshow("Test Camera", image)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# Create datasheet

In [119]:
prj_path = r"C:\Users\Chale\Desktop\Project ITM65\Model Framework\Code\Web-applications-for-elderly-physical-exercises\ElderFitmodel"

In [120]:
model_name = "handup_model"

In [121]:
class_name = "both_hand"

#create folder of the model
modelFolerPath = prj_path+"\Dataset/"+model_name
if not os.path.exists(modelFolerPath):
    os.makedirs(modelFolerPath)

In [122]:
# Absolute path of a file
go_src_path = prj_path+"\Dataset\predataset.csv"
go_dst_path = prj_path+"\Dataset\\"+model_name+"\predataset.csv"
data_path = prj_path+"\Dataset\\"+model_name+"\\"+class_name+".csv"

# Copying and rename the file
if not os.path.isfile(data_path):
    shutil.copy(go_src_path, go_dst_path)
    os.rename(go_dst_path, data_path)

# Data acquisition

In [123]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)
 
while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()
 
    # resizing the frame for better view
    frame = cv2.resize(frame, (880,720))
 
    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
 
    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True
 
    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 
    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    mp_drawing.draw_landmarks(
        image, 
        results.pose_landmarks, 
        mp_holistic.POSE_CONNECTIONS,            
    )
    
    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS
    )
    
    try:
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

        pose_row.insert(0, class_name)
            
        with open(data_path, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',')
            csv_writer.writerow(pose_row)
            
    except:
        cv2.putText(image, str('Error')
                    , (210,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    

    # Display the resulting image
    cv2.imshow("Test Data", image)
 
    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q') :
        break
 
# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()

# TRAIN MODEL

In [126]:
model_name = "handup_model"

In [127]:
# iterate over files in data directory
list_class = []
list_data = []
directory = modelFolerPath+"/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        list_data.append(pd.read_csv(f))
        x = filename.split(".")
        list_class.append(x[0])

In [131]:
sys.stdout.write("[  There is " + str(len(list_class)) + " class for this model  ]\n")
print("==============================================================================")
for i in list_class:
    print(i)

print("==============================================================================")
df_model = pd.concat(list_data,ignore_index=True)
print("df_model is now complete")

[  There is 4 class for this model  ]
both_hand
left_hand
normal
right_hand
df_model is now complete


In [132]:
df_model = df_model.drop(0)

In [133]:
df_model 

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1,both_hand,0.495241,0.641341,-0.815991,0.999954,0.514867,0.588570,-0.774460,0.999918,0.532227,...,0.247862,0.000103,0.589847,2.525757,-0.320869,0.000064,0.438337,2.544739,-0.309533,0.000092
2,both_hand,0.494693,0.642476,-0.790162,0.999958,0.514162,0.589906,-0.748717,0.999925,0.531386,...,0.350780,0.000124,0.590347,2.530525,-0.252592,0.000067,0.442775,2.550962,-0.183367,0.000102
3,both_hand,0.494449,0.642688,-0.698121,0.999961,0.513494,0.590607,-0.658345,0.999931,0.530067,...,0.392353,0.000125,0.595530,2.529970,-0.195906,0.000067,0.448439,2.553771,-0.137730,0.000102
4,both_hand,0.494614,0.643275,-0.698814,0.999965,0.513870,0.591256,-0.660191,0.999937,0.530466,...,0.344309,0.000122,0.591682,2.530621,-0.213101,0.000066,0.446596,2.553908,-0.188586,0.000099
5,both_hand,0.493950,0.642241,-0.700558,0.999968,0.513535,0.590976,-0.661660,0.999942,0.529988,...,0.266310,0.000122,0.591963,2.532448,-0.268450,0.000068,0.446722,2.555186,-0.283043,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8255,right_hand,0.502223,0.645100,-0.759471,0.999936,0.524775,0.590480,-0.715633,0.999885,0.543653,...,0.379918,0.000110,0.603500,2.600402,-0.337298,0.000067,0.448866,2.609451,-0.201858,0.000104
8256,right_hand,0.502391,0.645361,-0.817288,0.999938,0.524790,0.590503,-0.772855,0.999889,0.543557,...,0.378255,0.000105,0.599393,2.601093,-0.260209,0.000065,0.443917,2.609671,-0.211746,0.000101
8257,right_hand,0.502437,0.643419,-0.778318,0.999940,0.525169,0.588635,-0.735056,0.999892,0.543863,...,0.416624,0.000102,0.597140,2.601791,-0.221963,0.000064,0.442330,2.609216,-0.151561,0.000100
8258,right_hand,0.501058,0.639517,-0.676269,0.999942,0.524312,0.585448,-0.640222,0.999897,0.543348,...,0.388422,0.000100,0.591086,2.604295,-0.233405,0.000064,0.434828,2.609576,-0.173459,0.000099


In [134]:
df_model.to_csv("test.csv")

In [135]:
x = df_model .drop('class', axis=1) 
y = df_model ['class']

In [136]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=999)

In [137]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(), verbose = 1),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(), verbose = 1),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier(), verbose = 1),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier(), verbose = 1),
}

In [138]:
# list(pipelines.values())[3]
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())],
          verbose=1),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())],
          verbose=1),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())],
          verbose=1),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())],
          verbose=1)}

In [139]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing logisticregression, total=   0.2s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline] ... (step 2 of 2) Processing ridgeclassifier, total=   0.0s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing randomforestclassifier, total=   1.5s
[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing gradientboostingclassifier, total= 1.2min


In [140]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9993946731234867
gb 0.9993946731234867


In [141]:
model_dst=prj_path+"\Model/"
save_dst=model_dst+'handup_model.pkl'
pickle.dump(fit_models['rf'],open(save_dst, 'wb'))